<a href="https://colab.research.google.com/github/SPPPARK/Recidence_Dashborad/blob/main/Residence_Dashboard_ipynb_Draft_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0 Install package

In [1]:
!pip install streamlit pandas geopandas folium branca

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [2]:
!pip install streamlit_folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 5.4 MB/s eta 0:00:00


In [3]:
import requests
import pandas as pd
import plotly.graph_objects as go
from sqlalchemy import create_engine
import folium
from folium.plugins import MarkerCluster

Education Level

Income Level

Employment

House

# 2 Greate map


In [4]:
!pip install pyshp simpledbf

  Preparing metadata (setup.py) ... done
  Created wheel for simpledbf: filename=simpledbf-0.2.6-py3-none-any.whl size=13786 sha256=2ef665e69e381fabb0218887058c2a758cb8620e1d9f99ed0aad1d4c8e6f7e55
  Stored in directory: /root/.cache/pip/wheels/37/52/21/14be45b7c160488637e82d6a317f4379458bb4dd60be21d5fa
Successfully built simpledbf


In [5]:
!pip install streamlit

In [6]:
!pip install pyngrok

In [7]:
!pip install openai

In [8]:
%%writefile app.py

import streamlit as st
import pandas as pd
import geopandas as gpd
import folium
from folium import Choropleth
import json
import streamlit.components.v1 as components
from streamlit_folium import st_folium
import matplotlib.pyplot as plt

# Load your datasets here
data = pd.read_csv("/content/indexes_dataset.csv")
manual_mapping = pd.read_csv("/content/Completed_FineNeighborhood_to_CommunityAreaName.csv")
metadata = pd.read_csv("/content/indexes_metadata.csv")
feature_metadata = pd.read_csv("/content/metadata_with_index.csv")
index_metadata = pd.read_csv("/content/indexe_metadata.csv")

# Load local GeoJSON file
with open("/content/chicago_neighborhoods.geojson", "r", encoding="utf-8") as f:
    geojson_data = json.load(f)

# Index options and their factors
index_factors = {
    "wellness_index": ["EVI","SSI","EOI","CARDIO", "DIABETES", "RESPIR", "MHDIS", "CURSMOKE", "BINGEDRINK", "DRUGOD", "PHYINACT"],
    "EVI": ["AIRQUAL", "WATQUAL", "HEATVUL", "NOISEP"],
    "SSI": ["SCHLRAT", "HSDROP", "UNEMPLOY", "MEDINC"],
    "EOI": ["HOUSPRB", "HOUSBURD", "BROADND", "TRAFVOL", "COLLACC"]
}

# Sidebar selections
st.set_page_config(layout="wide")
st.sidebar.title("Index Configuration")
selected_index = st.sidebar.selectbox("Select Index", list(index_factors.keys()))
st.sidebar.subheader("Index Description")
index_description = metadata[metadata['index_name'] == selected_index]['description'].values
st.markdown(
    """
    <style>
        [data-testid="stSidebar"] {
            min-width: 400px;
            max-width: 400px;
        }
    </style>
    """,
    unsafe_allow_html=True
)

with st.sidebar.expander("ℹ️ Index Description", expanded=False):
    if len(index_description):
        st.markdown(
            f"<div style='font-size: 12px; text-align: justify;'>{index_description[0]}</div>",
            unsafe_allow_html=True
        )
    else:
        st.markdown("_No index-level description found._")

selected_factors = index_factors[selected_index]

with st.sidebar.expander("🧪 Factor Descriptions", expanded=True):
    rows = ""
    for factor in selected_factors:
        desc_row = metadata[metadata['index_name'] == factor]['description'].values
        desc_text = desc_row[0] if len(desc_row) else "(No description found)"
        rows += f"<tr><td><strong>{factor}</strong></td><td>{desc_text}</td></tr>"

    table_html = f"""
    <div style='font-size: 11px;text-align: justify;'>
        <table style='width: 100%; border-collapse: collapse;'>
            {rows}
        </table>
    </div>
    """
    st.markdown(table_html, unsafe_allow_html=True)

# Load GeoDataFrame
gdf_raw = gpd.GeoDataFrame.from_features(geojson_data["features"])
gdf_raw = gdf_raw.rename(columns={"pri_neigh": "FineNeighborhood"})
gdf_raw.set_crs(epsg=4326, inplace=True)

# Merge with manual mapping to get CommunityAreaName
gdf = gdf_raw.merge(manual_mapping, on="FineNeighborhood", how="left")

# Merge with index data
gdf = gdf.merge(data, on="CommunityAreaName", how="left")

# Ensure the CommunityAreaName exists in GeoJSON for rendering
gdf["CommunityAreaName"] = gdf["CommunityAreaName"].fillna("Unknown")

# Function to render map
def render_map(gdf, column, title):
    m = folium.Map(location=[41.8781, -87.6298], zoom_start=10, height=600)

    # Workaround: add CommunityAreaName to each feature property before converting to JSON
    gdf_json = json.loads(gdf.to_json())
    for idx, feature in enumerate(gdf_json["features"]):
        feature["properties"]["CommunityAreaName"] = gdf.iloc[idx]["CommunityAreaName"]

    Choropleth(
        geo_data=gdf_json,
        name="choropleth",
        data=gdf,
        columns=["CommunityAreaName", column],
        key_on="feature.properties.CommunityAreaName",
        fill_color="YlOrRd",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=title,
        highlight=True,
        nan_fill_color='gray'
    ).add_to(m)

    def style_function(feature):
        return {'fillColor': 'transparent','color': 'black','weight': 1,'fillOpacity': 0}

    def highlight_function(feature):
        return {'fillColor': '#ffff00','color': 'black','weight': 2,'fillOpacity': 0.3}

    tooltip_data = []
    for _, row in gdf.iterrows():
        name = row["CommunityAreaName"]
        value = row[column]
        value_str = f"{value:.2f}" if pd.notna(value) else "No data"
        tooltip_data.append({"tooltip": f"{name}  {value_str}"})

    tooltip_gdf = gdf.copy()
    tooltip_gdf["tooltip"] = [d["tooltip"] for d in tooltip_data]
    tooltip_gdf.set_crs(epsg=4326, inplace=True)

    folium.GeoJson(
        tooltip_gdf,
        name="Neighborhood Borders",
        style_function=style_function,
        highlight_function=highlight_function,
        tooltip=folium.GeoJsonTooltip(fields=["tooltip"], aliases=["Community: "])
    ).add_to(m)

    return m


# Main layout
st.markdown("""
    <style>
        iframe[title="streamlit_folium.st_folium"] {
            height: 600px !important;
            max-height: 600px !important;
            overflow: hidden;
        }
    </style>
""", unsafe_allow_html=True)
with st.container():
  st.title("Chicago Neighborhood Dashboard")
  st.markdown(f"#### Main Map: {selected_index}")
  main_map = render_map(gdf, selected_index, selected_index)
  st_folium(main_map, use_container_width=True,height=600)

# Panel maps with factor selectors on top
col1, col2 = st.columns(2)
with col1:
    st.markdown("<h4 style='text-align: center;'>Panel 1</h4>", unsafe_allow_html=True)
    st.markdown("<div style='font-size: 13px; color: gray;text-align: center;'>Select Panel 1 Factor</div>", unsafe_allow_html=True)
    factor1 = st.selectbox("", selected_factors, key="panel1")
    st.markdown(f"#### {factor1}")
    panel1_map_html = render_map(gdf, factor1, factor1)
    st_folium(panel1_map_html, use_container_width=True)

with col2:
    st.markdown("<h4 style='text-align: center;'>Panel 2</h4>", unsafe_allow_html=True)
    st.markdown("<div style='font-size: 13px; color: gray;text-align: center;'>Select Panel 2 Factor</div>", unsafe_allow_html=True)
    remaining_factors = [f for f in selected_factors if f != st.session_state.panel1]
    factor2 = st.selectbox("", remaining_factors, key="panel2")
    st.markdown(f"#### {factor2}")
    panel2_map_html = render_map(gdf, factor2, factor2)
    st_folium(panel2_map_html, use_container_width=True)

#Statistic Map
feature_desc = dict(zip(feature_metadata['column_name'], feature_metadata['description']))
index_desc = dict(zip(index_metadata['index_name'], index_metadata['index_description']))

def get_stats(data):
    return {'Min': data.min(), 'Q1': data.quantile(0.25), 'Mean': data.mean(), 'Q3': data.quantile(0.75), 'Max': data.max()}

def plot_full_stats(ax, data, title, chart_type='bar'):
    stats = get_stats(data)
    metrics, values = list(stats.keys()), list(stats.values())
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
    if chart_type == 'bar':
        bars = ax.bar(metrics, values, color=colors)
        for bar in bars:
            ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f"{bar.get_height():.2f}", ha='center', va='bottom')
    elif chart_type == 'box':
        ax.boxplot(data, vert=False, patch_artist=True, boxprops=dict(facecolor='#2ca02c'))
        stats_text = "\n".join([f"{k}: {v:.2f}" for k, v in stats.items()])
        ax.text(0.95, 0.95, stats_text, transform=ax.transAxes, ha='right', va='top', bbox=dict(facecolor='white', alpha=0.8))
        ax.set_yticks([])
    ax.set_title(title, fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.7)

def plot_dual_comparison(ax, feature_data, index_data, feature_name, index_name):
    feature_stats, index_stats = get_stats(feature_data), get_stats(index_data)
    metrics, colors = list(feature_stats.keys()), ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
    bars = ax.bar(metrics, feature_stats.values(), color=colors, alpha=0.7)
    ax2 = ax.twinx()
    line, = ax2.plot(metrics, index_stats.values(), color='black', marker='o', linewidth=2)
    ax.set_title(f"{feature_name} vs {index_name}", fontsize=12)
    ax.legend([bars[0], line], [feature_name, index_name], loc='upper left')

st.header("Feature Distributions")
box1, box2 = st.columns(2)
with box1:
    fig2, ax2 = plt.subplots(figsize=(8, 4))
    plot_full_stats(ax2, data[factor1], f"{factor1}\n{feature_desc.get(factor1, factor1)}", chart_type='box')
    st.pyplot(fig2)

with box2:
    fig3, ax3 = plt.subplots(figsize=(8, 4))
    plot_full_stats(ax3, data[factor2], f"{factor2}\n{feature_desc.get(factor2, factor2)}", chart_type='box')
    st.pyplot(fig3)

st.header("Feature vs Index Comparison")
comp1, comp2 = st.columns(2)
with comp1:
    fig4, ax4 = plt.subplots(figsize=(10, 5))
    plot_dual_comparison(ax4, data[factor1], data[selected_index], factor1, selected_index)
    st.pyplot(fig4)

with comp2:
    fig5, ax5 = plt.subplots(figsize=(10, 5))
    plot_dual_comparison(ax5, data[factor2], data[selected_index], factor2, selected_index)
    st.pyplot(fig5)
#Community Data ChatBot
st.markdown("---")
st.header("💬 Community Data Chatbot")
def auto_detect_column(df, keyword):
    return [col for col in df.columns if keyword.lower() in col.lower()][0]


def map_full_name_to_shortcut(user_input):
    user_input_lower = user_input.lower()
    desc_col = auto_detect_column(feature_metadata, 'description')
    name_col = auto_detect_column(feature_metadata, 'name')

    for _, row in feature_metadata.iterrows():
        if str(row[desc_col]).lower() in user_input_lower:
            return row[name_col]
    for _, row in feature_metadata.iterrows():
        if str(row[name_col]).lower() in user_input_lower:
            return row[name_col]
    return None


def search_answer(user_input):
    user_input_lower = user_input.lower()
    index_name_col = auto_detect_column(index_metadata, 'name')
    index_desc_col = auto_detect_column(index_metadata, 'description')
    feature_name_col = auto_detect_column(feature_metadata, 'name')
    feature_desc_col = auto_detect_column(feature_metadata, 'description')

    # --- Check for maximum ---
    if any(keyword in user_input_lower for keyword in ['highest', 'maximum', '最大', '最高']):
        shortcut_col = map_full_name_to_shortcut(user_input)
        if shortcut_col and shortcut_col in data.columns:
            neighborhood_col = auto_detect_column(data, 'community') if any(
                'community' in c.lower() for c in data.columns) else auto_detect_column(data, 'neigh')
            max_row = data.loc[data[shortcut_col].idxmax()]
            neighborhood_name = max_row[neighborhood_col]
            max_value = max_row[shortcut_col]
            return f"🏆 Among all communities, **{neighborhood_name}** has the highest **{shortcut_col}**, with a value of {max_value:.2f}."

    # --- Check for average ---
    if any(keyword in user_input_lower for keyword in ['average', 'mean', '平均']):
        shortcut_col = map_full_name_to_shortcut(user_input)
        if shortcut_col and shortcut_col in data.columns:
            avg_value = data[shortcut_col].mean()
            return f"📊 The average value of **{shortcut_col}** is {avg_value:.2f}."

    # --- Check for definition ---
    for _, row in index_metadata.iterrows():
        if str(row[index_name_col]).lower() in user_input_lower:
            return f"📈 {row[index_name_col]}: {row[index_desc_col]}"

    for _, row in feature_metadata.iterrows():
        if str(row[feature_name_col]).lower() in user_input_lower:
            return f"🔎 {row[feature_name_col]}: {row[feature_desc_col]}"

    return "❓ Sorry, I couldn't understand your question. Please try asking differently."


st.markdown("""
    <style>
    .chat-bubble {
      background-color: #f0f2f6;
      padding: 12px;
      border-radius: 10px;
      margin-bottom: 10px;
      display: inline-block;
      max-width: 90%;
      color: black;
    }
    .user-bubble {
      background-color: #d1e7dd;
      text-align: right;
      margin-left: auto;
      color: black;
    }
    </style>
    """, unsafe_allow_html=True)

user_input = st.text_input("Enter your question (e.g., Which community has the highest HOUSPRB?):",
                            key="chatbot_input")

if user_input:
    st.markdown(f'<div class="chat-bubble user-bubble">{user_input}</div>', unsafe_allow_html=True)
    response = search_answer(user_input)
    st.markdown(f'<div class="chat-bubble">{response}</div>', unsafe_allow_html=True)

Writing app.py


In [9]:
# 设置 ngrok authtoken
from pyngrok import ngrok
ngrok.set_auth_token("2vh8RdP8gH4DZ4IPG49jCTHtQkD_8766JsxXk3kHKzYxUJxY6")

# 启动 ngrok 隧道映射本地 8501 端口
public_url = ngrok.connect("8501")
print("Streamlit app public URL:", public_url)

Streamlit app public URL: NgrokTunnel: "https://5004-34-16-44-110.ngrok-free.app" -> "http://localhost:8501"


In [10]:
# 在后台启动 Streamlit 应用
!streamlit run app.py &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.44.110:8501

2025-05-04 20:40:36.627 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2025-05-04 20:40:39.766 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2025-05-04 20:40:47.876 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2025-05-04 20:40:50.860 `label` got an empty value. This is discouraged for accessibility reasons and may be disal